In [115]:
import streamlit as st
import urllib.request
import urllib.parse
import pandas as pd
import json
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [116]:
def getID():
    with open("memo.txt", 'r') as f:
        lines = f.readlines()
        client_id = lines[0].strip()  # 첫 번째 줄
        client_secret = lines[1].strip()  # 두 번째 줄

    return client_id, client_secret

In [155]:
web_df = pd.DataFrame(columns = ("Title", "link", "postdate","Description"))

keyword = input("검색할 키워드를 입력하세요!")
count = int(input("보고싶은 뉴스의 수를 입력하세요!"))

if not keyword:
    print("검색할 키워드를 입력해주세요!!")
else:
    client_id, client_secret = getID()
    encText = urllib.parse.quote(keyword)
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display={count}"  # JSON 결과 URL
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        
        remove_tag = re.compile('<.*?>') # html 태그 제거 정규식
        
        for item in items:
            title = re.sub(remove_tag, '', item['title'])  # 태그 제거
            link = item['link']
            postdate = item['postdate']
            postdate = pd.to_datetime(postdate, format='%Y%m%d').strftime('%Y년 %m월 %d일')

            info_req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
            info_html = urlopen(info_req).read()
            bs_obj = BeautifulSoup(info_html, 'html.parser')
            
            iframe = bs_obj.find('iframe', id='mainFrame')
            description = []
            if iframe:
                iframe_src = iframe.get('src')
                iframe_url = "https://blog.naver.com" + iframe_src  # 네이버 블로그는 절대 경로 사용
                
                # 두 번째 요청: iframe의 URL에서 본문 가져오기
                iframe_req = Request(iframe_url, headers={'User-Agent': 'Mozilla/5.0'})
                iframe_html = urlopen(iframe_req).read()
                iframe_soup = BeautifulSoup(iframe_html, 'html.parser')
                
                # 본문 컨테이너 찾기
                content = iframe_soup.find('div', {'class': 'se-main-container'})  # 스마트에디터 본문
                if content:
                    description.append(content.get_text(strip=True))
                else:
                    print("본문을 가져올 수 없습니다. HTML 구조를 확인하세요.")
            else:
                # 스마트에디터 구조 (최신 블로그)
                content = None
                
                if bs_obj.find('div', {'class': 'se-main-container'}):
                    content = bs_obj.find('div', {'class': 'se-main-container'})
                elif bs_obj.find('div', {'id': 'postViewArea'}):  # 이전 블로그 구조 (구버전 블로그)
                    content = bs_obj.find('div', {'id': 'postViewArea'})
                elif bs_obj.find('div', {'class': 'contents_style'}):  # 다른 경우 (필요시 추가)
                    content = bs_obj.find('div', {'class': 'contents_style'})
                
                # 본문 내용 추출
                if content:
                    full_text = content.get_text(strip=True)  # 텍스트만 추출하고 공백 제거
                    description.append(full_text)
                else:
                    print("본문을 찾을 수 없습니다. HTML 구조를 확인하세요.")
            
            web_df.loc[len(web_df)] = [title, link, postdate,description]  # 데이터프레임 추가
        # 리스트 형태의 Description을 문자열로 병합하고 \u200b 제거
        web_df['Description'] = web_df['Description'].apply(lambda x: ' '.join(x).replace('\u200b', '') if isinstance(x, list) else x)
    else:
        print(f"Error Code: {rescode}")

검색할 키워드를 입력하세요! 손흥민
보고싶은 뉴스의 수를 입력하세요! 3


In [140]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

In [163]:
for i in range(len(web_df)):
    article = web_df['Description'][i]
    article = '\n'.join([line.strip() + '.' for line in text_lines if line.strip()])
    parser = PlaintextParser.from_string(article, Tokenizer("korean"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 3)  # 요약할 문장 개수
    for sentence in summary:
        print(sentence)
    print('---')

NameError: name 'text_lines' is not defined

In [162]:
web_df['Description'][2]

"홍명보 감독이 이끄는 남자 축구국가대표팀(FIFA랭킹 22위)은 19일 오후 11시(한국시간) 요르단의암만 국제경기장에서 열린팔레스타인(100위)과의 2026 국제축구연맹(FIFA) 북중미월드컵 아시아 3차 예선B조 6차전에서손흥민(토트넘)의 골에 힘입어 1-1무승부를 거뒀다.(대한축구협회 제공)홍명보 감독은 쿠웨이트와의 5차전(3-1승) 선발 라인업을 변동 없이 기용했다.4-2-3-1 포메이션을 기준으로 최전방에오세훈(마치다 젤비아)를 세우고 2선에손흥민-이재성(마인츠)-이강인(파리생제르맹)을 배치했다.중원에황인범(페예노르트)과박용우(알아인)를 세워 공격과 수비의 연결고리 역할을 맡겼고,포백은이명재(울산 HD)-김민재(뮌헨)-조유민(샤르자)-설영우(즈베즈다)로 꾸렸다.골문은 변함없이조현우(울산 HD)가 지켰다.전력 차이가 많이 났지만 경기 초반 먼저 골을 터뜨린 쪽은팔레스타인이었다.상대 선수와 경합을 펼치고 있는 김민재의 모습.(대한축구협회 제공)전반 12분중앙수비수김민재가 골키퍼조현우에게 건넨 땅볼 백패스가 짧게 처리되며 오히려자이드 쿤바르에게 1대1 찬스를 내줬고, 쿤바르가 조현우를 따돌리고 비어 있는 골대에 볼을 차 넣었다.일격을 당했지만 한국은 흔들리지 않았고, 4분 만에1-1 균형을 맞췄다.손흥민이 동점골을 터뜨린 뒤 세리머니를 하고 있다.(대한축구협회 제공)전반 16분이명재가 내준 볼을 페널티 아크 부근에서 잡은이재성이 곧바로 문전으로 침투하던손흥민에게 원터치로 패스를 내줬고 이를 받은 손흥민이 반대편 골문으로 정교한 오른발 슈팅으로 동점골을 터뜨리며 자신의 131번째 A매치에서 통산51호골을 작성했다.황선홍대전하나시티즌 감독(50골)을 밀어내고통산 득점부문단독 2위로 올라섰으며 역대 1위차범근전 축구대표팀 감독(58골)과의 격차는 7골로 좁혔다.뿐만 아니라 올 한해A매치에서10번째 득점을 기록하며 이 부문개인 최다기록을 새로 썼다. 지난2015년자신이 세운종전 기록(9골)을 스스로 뛰어넘어 최초로 두 자릿수 득점의 주인공이 됐다.손흥민의 동점골